In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
from birdclef.utils import get_spark

spark = get_spark(cores=16, memory="20g")
df = spark.read.parquet(
    "../data/processed/birdclef-2023/train_embeddings/consolidated_v3"
)
df.printSchema()

root
 |-- species: string (nullable = true)
 |-- track_stem: string (nullable = true)
 |-- track_type: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- embedding: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- prediction_vec: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- predictions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- rank: long (nullable = true)
 |    |    |-- index: long (nullable = true)
 |    |    |-- label: string (nullable = true)
 |    |    |-- mapped_label: string (nullable = true)
 |    |    |-- probability: double (nullable = true)
 |-- start_time: long (nullable = true)
 |-- energy: double (nullable = true)



In [3]:
from pyspark.sql import Window, functions as F

# keep the track_type for the highest energy
highest_energy_channel = (
    df
    # get the track stem without the part
    .withColumn("original_track_stem", F.split(F.col("track_stem"), "_").getItem(0))
    .where("track_type != 'original'")
    # get the track type that has the most energy
    .withColumn(
        "rank",
        F.rank().over(
            Window.partitionBy("original_track_stem").orderBy(F.desc("energy"))
        ),
    )
    # keep the first row
    .where(F.col("rank") == 1)
    # drop the rank column
    .select("species", "track_stem", "track_type")
    .distinct()
)

# get the highest predictions by exploding the values
exploded_embeddings = (
    df
    # join against the highest energy channel
    .join(
        highest_energy_channel,
        on=["species", "track_stem", "track_type"],
        how="inner",
    )
    # explode the embeddings, these are ordered by confidence
    .withColumn("predictions", F.explode("predictions")).select(
        "species",
        "track_stem",
        "track_type",
        "start_time",
        "track_name",
        "embedding",
        "predictions.*",
    )
    # simplifying assumption: we assume the prediction with the highest confidence is the true label
    .where("rank = 0")
).cache()

exploded_embeddings.drop("embedding").show(n=5)

+-------+----------+----------+----------+--------------------+----+-----+--------------------+------------+--------------------+
|species|track_stem|track_type|start_time|          track_name|rank|index|               label|mapped_label|         probability|
+-------+----------+----------+----------+--------------------+----+-----+--------------------+------------+--------------------+
|abythr1|  XC233199|   source0|        15|abythr1/XC233199_...|   0| 3185|Turdus leucomelas...|     pabthr1|  0.2241482138633728|
|abythr1|  XC233199|   source0|         6|abythr1/XC233199_...|   0| 3021|Terpsiphone virid...|     afpfly1|0.040495436638593674|
|abythr1|  XC233199|   source0|        18|abythr1/XC233199_...|   0| 3164|Turdus abyssinicu...|     abythr1|0.040056031197309494|
|abythr1|  XC233199|   source0|         9|abythr1/XC233199_...|   0| 1151|Erpornis zanthole...|     whbyuh1| 0.15738973021507263|
|abythr1|  XC233199|   source0|        54|abythr1/XC233199_...|   0| 3185|Turdus leucomela

In [4]:
# quick count of the number of samples
counts = (
    exploded_embeddings.groupBy("species")
    .agg(F.count("*").alias("n"))
    .orderBy(F.desc("n"))
)
counts.show(n=5)
counts.orderBy("n").show(n=5)

+-------+-----+
|species|    n|
+-------+-----+
|thrnig1|12987|
| wlwwar| 9249|
|combuz1| 7173|
| hoopoe| 6731|
| barswa| 6191|
+-------+-----+
only showing top 5 rows

+-------+---+
|species|  n|
+-------+---+
|afpkin1|  3|
|whhsaw1|  4|
|whctur2|  4|
|golher1|  5|
|lotlap1|  8|
+-------+---+
only showing top 5 rows



In [5]:
rarity_min_count = 100
rare_species_count = (
    exploded_embeddings.groupBy("species")
    .agg(F.count("*").alias("n"))
    .where(f"n < {rarity_min_count}")
)
rare_species_count.show(n=5)

+-------+---+
|species|  n|
+-------+---+
|purgre2| 60|
|bubwar2| 90|
|rehwea1| 69|
|kvbsun1| 80|
|equaka1| 63|
+-------+---+
only showing top 5 rows



In [6]:
# if there are a lot of examples, we can use a higher threshold
common_species = exploded_embeddings.where("probability > 0.4").join(
    rare_species_count.select("species"), on="species", how="left_anti"
)
# these ones are less common so we use a lower threshold so we have at least one
# example for each species
rare_species = exploded_embeddings.where("probability > 0.05").join(
    rare_species_count.select("species"), on="species", how="inner"
)
prepared = common_species.union(rare_species).select(
    "species", "probability", "embedding"
)
prepared.show(n=5)
prepared.count()

+-------+------------------+--------------------+
|species|       probability|           embedding|
+-------+------------------+--------------------+
|afghor1| 0.511886715888977|[1.00166213512420...|
|afghor1|0.9965255856513977|[0.57833033800125...|
|afghor1|0.9904100894927979|[0.61484068632125...|
|afghor1|0.9988522529602051|[1.26016914844512...|
|afghor1|0.5952618718147278|[0.75000149011611...|
+-------+------------------+--------------------+
only showing top 5 rows



74490

In [7]:
df.select("species").distinct().count()

264

In [8]:
# lets check that we have the right number of classes, and how many examples we are working with
prepared_counts = (
    prepared.groupBy("species").agg(F.count("*").alias("n")).orderBy(F.desc("n"))
)
print(f"number of species {prepared_counts.count()}")

prepared_counts.show(n=5)
prepared_counts.orderBy("n").show(n=5)

number of species 264
+-------+----+
|species|   n|
+-------+----+
|thrnig1|3833|
| hoopoe|3822|
|eubeat1|3116|
| wlwwar|2687|
| barswa|2603|
+-------+----+
only showing top 5 rows

+-------+---+
|species|  n|
+-------+---+
|afpkin1|  2|
|whctur2|  2|
|rehblu1|  2|
|whhsaw1|  3|
|easmog1|  4|
+-------+---+
only showing top 5 rows



In [9]:
# percentage of clips that have a prediction of any kind
prepared.count() / exploded_embeddings.count() * 100

35.7261046603646

base classifiers

In [10]:
data = prepared.toPandas()
data.head()

,species,probability,embedding
0,afghor1,0.511887,"[1.0016621351242065, 1.2551445960998535, 0.242..."
1,afghor1,0.996526,"[0.5783303380012512, 1.845029354095459, 0.2178..."
2,afghor1,0.990410,"[0.6148406863212585, 1.5936590433120728, 0.504..."
3,afghor1,0.998852,"[1.2601691484451294, 2.366661787033081, 0.2103..."
4,afghor1,0.595262,"[0.7500014901161194, 1.3813127279281616, 0.219..."


In [11]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)


def model_eval(truth, preds):
    print("Accuracy:", accuracy_score(truth, preds))
    print(
        "Precision:",
        precision_score(truth, preds, average="macro"),
    )
    print(
        "Recall:",
        recall_score(truth, preds, average="macro"),
    )
    print(
        "F1 Score:",
        f1_score(truth, preds, average="macro"),
    )


train_x, test_x, train_y, test_y = train_test_split(
    np.stack(data["embedding"]),
    data["species"],
    test_size=0.33,
    stratify=data["species"],
)

In [11]:
# check how long it takes to train linearsvc vs svc
from sklearn.svm import LinearSVC, SVC

# NOTE: this doesnt have an option to dump out probabilities
# print("linearsvc")
# %time clf = LinearSVC().fit(train_x, train_y)
# model_eval(clf.predict(test_x), test_y)

# CPU times: total: 2min 15s
# Wall time: 2min 15s
# Accuracy: 0.9044829550077292
# Precision: 0.7786332701592122
# Recall: 0.7993822579967949
# F1 Score: 0.7792748609128219

# setting probabilities makes this _much_ slower than it needs to be
print("svc")
%time clf = SVC(probability=True).fit(train_x, train_y)
%time model_eval(clf.predict(test_x), test_y)

# without pca
# CPU times: total: 11min 50s
# Wall time: 11min 54s
# Accuracy: 0.9250264421121146
# Precision: 0.6870300689036098
# Recall: 0.8614117101411818
# F1 Score: 0.7402244935237366

svc
CPU times: total: 11min 50s
Wall time: 11min 54s
Accuracy: 0.9250264421121146
Precision: 0.6870300689036098
Recall: 0.8614117101411818
F1 Score: 0.7402244935237366


f:\code\kaggle\birdclef-2023\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
# use a pipeline to setup pca
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

print("svc with pca")
%time clf = Pipeline([("pca", PCA(n_components=0.90)), ("svc", SVC(probability=True))]).fit(train_x, train_y)
%time model_eval(clf.predict(test_x), test_y)

# svc with pca
# CPU times: total: 9min 32s
# Wall time: 10min 22s

svc with pca
CPU times: total: 9min 32s
Wall time: 10min 22s


In [12]:
from sklearn.linear_model import LogisticRegression

print("logistic regression")
%time clf = LogisticRegression(solver="newton-cholesky").fit(train_x, train_y)
%time model_eval(clf.predict(test_x), test_y)

# CPU times: total: 29min 35s
# Wall time: 4min 21s
# Accuracy: 0.9232365145228216
# Precision: 0.7666584388212514
# Recall: 0.8696996457554622
# F1 Score: 0.8028503796054116
# CPU times: total: 531 ms
# Wall time: 1.99 s

logistic regression
CPU times: total: 29min 35s
Wall time: 4min 21s
Accuracy: 0.9232365145228216
Precision: 0.7666584388212514
Recall: 0.8696996457554622
F1 Score: 0.8028503796054116
CPU times: total: 531 ms
Wall time: 1.99 s


f:\code\kaggle\birdclef-2023\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
clf.steps[0][1].components_.shape

(204, 320)

In [24]:
print("logistic regression with pca")
pipeline = Pipeline(
    [
        ("pca", PCA(n_components=0.95)),
        ("lr", LogisticRegression(solver="newton-cholesky")),
    ]
)
%time clf = pipeline.fit(train_x, train_y)
%time model_eval(clf.predict(test_x), test_y)

# logistic regression with pca 0.90
# CPU times: total: 13min 16s
# Wall time: 2min 6s
# Accuracy: 0.9162802050280693
# Precision: 0.7555922870382019
# Recall: 0.8559744316050081
# F1 Score: 0.7887907714809498
# dims = (204, 320)

# logistic regression with pca 0.95
# CPU times: total: 17min 17s
# Wall time: 3min 5s
# Accuracy: 0.9201855015865267
# Precision: 0.7663412257014655
# Recall: 0.8615520243852796
# F1 Score: 0.7988655932230133
# CPU times: total: 1.03 s
# Wall time: 397 ms
# dims = (250, 320)

logistic regression with pca
CPU times: total: 17min 17s
Wall time: 3min 5s
Accuracy: 0.9201855015865267
Precision: 0.7663412257014655
Recall: 0.8615520243852796
F1 Score: 0.7988655932230133
CPU times: total: 1.03 s
Wall time: 397 ms


f:\code\kaggle\birdclef-2023\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
clf.steps[0][1].components_.shape

(250, 320)

In [27]:
# create a pipeline where we feature scale, then pca, then train
from sklearn.preprocessing import StandardScaler

print("logistic regression with pca")
pipeline = Pipeline(
    [
        ("scale", StandardScaler()),
        # ("pca", PCA(n_components=0.95)),
        ("lr", LogisticRegression(solver="saga")),
    ]
)
%time clf = pipeline.fit(train_x, train_y)
%time model_eval(clf.predict(test_x), test_y)

# CPU times: total: 14min 58s
# Wall time: 15min 9s
# Accuracy: 0.918192173134814
# Precision: 0.753662741892275
# Recall: 0.8684001488280116
# F1 Score: 0.7937295973057623
# CPU times: total: 641 ms
# Wall time: 366 ms

logistic regression with pca


f:\code\kaggle\birdclef-2023\venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


CPU times: total: 14min 58s
Wall time: 15min 9s
Accuracy: 0.918192173134814
Precision: 0.753662741892275
Recall: 0.8684001488280116
F1 Score: 0.7937295973057623
CPU times: total: 641 ms
Wall time: 366 ms


f:\code\kaggle\birdclef-2023\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
from skopt import BayesSearchCV
from sklearn.svm import SVC

cv_model = BayesSearchCV(
    SVC(probability=True),
    {
        "C": (1e-6, 1e6, "log-uniform"),
        "gamma": (1e-6, 1e1, "log-uniform"),
        "kernel": ["rbf", "linear"],
    },
    n_iter=32,
    scoring="precision_macro",
    verbose=4,
    cv=3,
    n_points=8,
    n_jobs=-1,
)

cv_model.fit(train_x, train_y)
print(cv_model.best_params_)
model_eval(test_y, cv_model.predict(test_x))

Fitting 3 folds for each of 8 candidates, totalling 24 fits


f:\code\kaggle\birdclef-2023\venv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Fitting 3 folds for each of 8 candidates, totalling 24 fits


f:\code\kaggle\birdclef-2023\venv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Fitting 3 folds for each of 8 candidates, totalling 24 fits


f:\code\kaggle\birdclef-2023\venv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Fitting 3 folds for each of 8 candidates, totalling 24 fits


f:\code\kaggle\birdclef-2023\venv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


OrderedDict([('C', 414521.9791438899), ('gamma', 7.245553401489778e-06), ('kernel', 'rbf')])
Accuracy: 0.9370677731673582
Precision: 0.8922478213125218
Recall: 0.8216002023315834
F1 Score: 0.8442900853568304


f:\code\kaggle\birdclef-2023\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Saving models to disk

In [16]:
import pickle
from pathlib import Path

pickle.dump(
    cv_model,
    Path("../data/models/baseline/svc-opt-v1.pkl").open("wb"),
)